In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import teradatasql
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import date
pd.set_option("display.max_columns", None)
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [2]:
pd.pandas.set_option('display.max_columns',None)

In [3]:
con = teradatasql.connect(host='XXXX')
cur = con.cursor()
consulta= cur.execute('select * from DataLake sample 75000 where Clase = 1')
des = consulta.description
datos = consulta.fetchall()
dfa = pd.DataFrame.from_records(datos,columns = [i[0] for i in des])

In [4]:
con = teradatasql.connect(host='XX', user='XX', password='XX')
cur = con.cursor()
consulta= cur.execute('select * from D_cientificos_datos.FT_Input_modelo_V2 sample 75000 where Clase = 0')
des = consulta.description
datos = consulta.fetchall()
dfb = pd.DataFrame.from_records(datos,columns = [i[0] for i in des])

In [5]:
# unir dos dataframe
df=pd.concat([dfa, dfb])

In [6]:
df.shape

(150000, 94)

In [58]:
# Conexión CSV

In [12]:
df.to_csv("Modelo_pca.csv")

In [13]:
df1 = df.copy()

In [14]:
## Elinimo variables que no funcionan en el modelo

In [15]:
df1.drop(labels=["Nro_Terminal","Q_Dias_hasta_falla","Q_tx_Rech_Ult4D","BonXPL","estado_terminal","Q_tx_Rech_Ult2D","Marca_no_Newland","Adm_Diners","superestado_terminal","BonHos","Adm_American_Express","Operatoria_no_Legacy"], axis=1, inplace=True)

## Elimino Variables poco relevantes

In [350]:
#df1.drop(labels=["Nro_Terminal","Q_Dias_hasta_falla","Facturacion_sin_descuento","Facturacion_con_descuento","BonHos","BonXPL","Desc_Agrup_Reducida_Rubro_Bain","Desc_Agrupacion_Rubro_Bain","Acepta_liquidacion","cod_plan_Ater","Cod_plan_host","Sub_segmento_corporativo_2","cod_rubro","Localidad","CUIT_VIP","Version_Origen","Marca","Segmento","Q_tx_Rech_Ult15D","Q_tx_Aprob_Ult2D","Prop_Vol_tx_Rech_Ult2D","Vol_tx_Rech_Ult10D"], axis=1, inplace=True)

## Elimino No Numericas

In [351]:
#df1.drop(labels=["Tipo_inactividad","serv_tecnico","Provincia","tipo_tecnologia","estado_terminal","superestado_terminal","Modelo","Plantilla_Origen","Medio","Condicion_Soft","tipo_recambio","Tipo_Utilidad_Tecnologia"],axis=1, inplace=True)

In [352]:
# Variables poco relevantes
# df1.drop(labels=['CUIT_VIP','Adm_Diners','Adm_American_Express','superestado_terminal','Operatoria_no_Legacy','Adm_Posnet','Modelo','Marca_no_Newland','Tipo_inactividad','Prop_Vol_tx_Rech_UltD','Prioridad_por_rubro','Q_tx_Rech_Ult2D','Q_tx_Rech_UltD','Tipo_Utilidad_Tecnologia','cod_rubro','Q_tx_Aprob_Ult3D','estado_terminal','Prop_Vol_tx_Rech_Ult3D','Porcentaje_Bonificacion_Teorico','Q_tx_Rech_Ult15D','Q_tx_Aprob_Ult2D','Prop_Vol_tx_Rech_Ult2D','cod_plan_Ater','Cod_plan_host','Localidad','Vol_tx_Rech_Ult10D','Sub_segmento_corporativo_2'], axis=1, inplace=True)

In [16]:
df1.shape

(150000, 82)

In [17]:
df1.head(5)

,Periodo,Antiguedad,Tipo_inactividad,serv_tecnico,cod_rubro,Provincia,Localidad,tipo_tecnologia,Banco_Visa,CUIT_VIP,Acepta_liquidacion,cod_plan_Ater,Cod_plan_host,Facturacion_con_descuento,Facturacion_sin_descuento,Porcentaje_Bonificacion_Teorico,Desc_Agrupacion_Rubro_Bain,Desc_Agrup_Reducida_Rubro_Bain,Modelo,Plantilla_Origen,Version_Origen,Medio,Marca,Condicion_Soft,Segmento,Sub_segmento_corporativo_2,Operatoria_no_Newland,No_newland,Adm_Posnet,Adm_Visa_Argentina,Adm_Prisma,tipo_recambio,Prioridad_por_volumen,Prioridad_por_rubro,Tipo_Utilidad_Tecnologia,Q_tx_Aprob_UltM,Q_tx_Rech_UltM,Prop_Q_tx_Rech_UltM,Q_tx_Aprob_Ult15D,Q_tx_Rech_Ult15D,Prop_Q_tx_Rech_Ult15D,Q_tx_Aprob_Ult10D,Q_tx_Rech_Ult10D,Prop_Q_tx_Rech_Ult10D,Q_tx_Aprob_Ult5D,Q_tx_Rech_Ult5D,Prop_Q_tx_Rech_Ult5D,Q_tx_Aprob_Ult4D,Prop_Q_tx_Rech_Ult4D,Q_tx_Aprob_Ult3D,Q_tx_Rech_Ult3D,Prop_Q_tx_Rech_Ult3D,Q_tx_Aprob_Ult2D,Prop_Q_tx_Rech_Ult2D,Q_tx_Aprob_UltD,Q_tx_Rech_UltD,Prop_Q_tx_Rech_UltD,Vol_tx_Aprob_UltM,Vol_tx_Rech_UltM,Prop_Vol_tx_Rech_UltM,Vol_tx_Aprob_Ult15D,Vol_tx_Rech_Ult15D,Prop_Vol_tx_Rech_Ult15D,Vol_tx_Aprob_Ult10D,Vol_tx_Rech_Ult10D,Prop_Vol_tx_Rech_Ult10D,Vol_tx_Aprob_Ult5D,Vol_tx_Rech_Ult5D,Prop_Vol_tx_Rech_Ult5D,Vol_tx_Aprob_Ult4D,Vol_tx_Rech_Ult4D,Prop_Vol_tx_Rech_Ult4D,Vol_tx_Aprob_Ult3D,Vol_tx_Rech_Ult3D,Prop_Vol_tx_Rech_Ult3D,Vol_tx_Aprob_Ult2D,Vol_tx_Rech_Ult2D,Prop_Vol_tx_Rech_Ult2D,Vol_tx_Aprob_UltD,Vol_tx_Rech_UltD,Prop_Vol_tx_Rech_UltD,Clase
0,202102,1174,Activo,Concesionaria Siscard,5812,CORDOBA,CORDOBA,WIFI,7.0,NO,SI,Q3,CP,880.00,880.00,0.00,Restaurants & Food,Restaurants & Food,S920,VP037,0218P17,GPRS,AMEX,Cambiar Soft,Small,10M-25M,0,0,0,0,0,Newland,3,3,Util,831,32,0.0371,436,17,0.0375,297,6,0.0198,144,2,0.0137,127,0.0155,111,2,0.0177,81,0.0241,48,0,0.0000,2268673.0000,88370.0000,0.03749189,1178055.0000,49750.0000,0.04051946,818765.0000,17410.0000,0.02082100,411729.0000,7500.0000,0.01788998,353879.0000,7500.0000,0.02075383,320029.0000,7500.0000,0.02289874,251779.0000,7500.0000,0.02892637,150359.0000,0.0000,0E-8,1
1,202012,110,Activo,Concesionaria Siscard,5411,SANTA FE,ROSARIO,GSM,67.0,NO,NO,Q3,CP,None,None,0.00,Supermarkets & food provider,Supermarkets & food provider,S920,VP036,0213P24,GPRS,POS INTEGRADO,Cambiar Soft,Small,1M-3M,1,1,0,1,0,NA Recambio,5,2,Util,30,7,0.1892,20,6,0.2308,15,1,0.0625,0,0,0.0000,0,0.0000,0,0,0.0000,0,0.0000,0,0,0.0000,12050.0000,3330.0000,0.21651495,5950.0000,2050.0000,0.25625000,5320.0000,750.0000,0.12355848,0.0000,0.0000,0E-8,0.0000,0.0000,0E-8,0.0000,0.0000,0E-8,0.0000,0.0000,0E-8,0.0000,0.0000,0E-8,1
2,202103,1235,Activo,Concesionaria Siscard,5411,SANTA FE,ROSARIO,WIFI,67.0,NO,NO,E7,CP,880.00,880.00,0.00,Supermarkets & food provider,Supermarkets & food provider,S920,VP037,0218P29,Ethernet/Wi-Fi,AMEX,Cambiar Soft,Small,3M-5M,0,0,0,0,0,NA Recambio,4,2,Util,68,2,0.0286,68,2,0.0286,68,2,0.0286,13,0,0.0000,0,0.0000,0,0,0.0000,0,0.0000,0,0,0.0000,60498.0000,3215.0000,0.05046066,60498.0000,3215.0000,0.05046066,60498.0000,3215.0000,0.05046066,10931.0000,0.0000,0E-8,0.0000,0.0000,0E-8,0.0000,0.0000,0E-8,0.0000,0.0000,0E-8,0.0000,0.0000,0E-8,1
3,202103,33,Sin uso,SIENCOM,5571,MENDOZA,VILLA NUEVA DE GUAYMALLEN,SIN DATO,72.0,NO,SI,AB,AB,None,None,0.00,Autoparts & Motorcycles,Autoparts & Motorcycles,S920,VP037,0218P17,Ethernet/Wi-Fi,CTLS,Cambiar Soft,Small,1M-3M,0,0,0,0,0,NA Recambio,5,7,Util,0,1,1.0000,0,0,0.0000,0,0,0.0000,0,0,0.0000,0,0.0000,0,0,0.0000,0,0.0000,0,0,0.0000,0.0000,5800.0000,1.00000000,0.0000,0.0000,0E-8,0.0000,0.0000,0E-8,0.0000,0.0000,0E-8,0.0000,0.0000,0E-8,0.0000,0.0000,0E-8,0.0000,0.0000,0E-8,0.0000,0.0000,0E-8,1
4,202103,81,Activo,Concesionaria Siscard,5641,SANTA FE,ROSARIO,WIFI,17.0,NO,SI,SQ,SQ,None,None,0.00,Retail - clothing,Retail - clothing,S920,VP037,0218P29,Ethernet/Wi-Fi,AMEX,Cambiar Soft,Micro - Merchant,750K-1M,0,0,0,0,0,OK - Util,5,4,Util,18,0,0.0000,10,0,0.0000,3,0,0.0000,0,0,0.0000,0,0.0000,0,0,0.0000,0,0.0000,0,0,0.0000,30380.0000,0.0000,0E-8,15420.0000,0.0000,0E-8,3980.0000,0.0000,0E-

## Revisamos y Eliminamos las N.A.

In [18]:
df1.isnull().sum()

Periodo                     0
Antiguedad                  0
Tipo_inactividad            0
serv_tecnico                0
cod_rubro                 165
                         ... 
Prop_Vol_tx_Rech_Ult2D      0
Vol_tx_Aprob_UltD           0
Vol_tx_Rech_UltD            0
Prop_Vol_tx_Rech_UltD       0
Clase                       0
Length: 82, dtype: int64

In [ ]:
df1.isnull().mean().head(50)

In [19]:
df1.dropna(axis=0,inplace=True)

In [21]:
df1.shape

(96917, 82)

## Pasamos a numericos todos los valores

In [359]:
# TODOS LOS CAMPOS = 
#campos_no_numericos = ["Tipo_inactividad","serv_tecnico","Provincia","Localidad","tipo_tecnologia","estado_terminal","superestado_terminal","CUIT_VIP","Plantilla_Origen","Version_Origen","Acepta_liquidacion","Version_Origen","Medio","Marca","Condicion_Soft","Segmento","Sub_segmento_corporativo_2","Tipo_Utilidad_Tecnologia","tipo_recambio","cod_plan_Ater","Cod_plan_host","Desc_Agrupacion_Rubro_Bain","Desc_Agrup_Reducida_Rubro_Bain","Modelo"]


In [23]:
campos_no_numericos = ["Tipo_inactividad","serv_tecnico","Provincia","Localidad","tipo_tecnologia","CUIT_VIP","Plantilla_Origen","Version_Origen","Acepta_liquidacion","Version_Origen","Medio","Marca","Condicion_Soft","Segmento","Sub_segmento_corporativo_2","Tipo_Utilidad_Tecnologia","tipo_recambio","cod_plan_Ater","Cod_plan_host","Desc_Agrupacion_Rubro_Bain","Desc_Agrup_Reducida_Rubro_Bain","Modelo"]

In [24]:
df1[campos_no_numericos] = df1[campos_no_numericos].apply(LabelEncoder().fit_transform)

In [25]:
df1.head(2)

,Periodo,Antiguedad,Tipo_inactividad,serv_tecnico,cod_rubro,Provincia,Localidad,tipo_tecnologia,Banco_Visa,CUIT_VIP,Acepta_liquidacion,cod_plan_Ater,Cod_plan_host,Facturacion_con_descuento,Facturacion_sin_descuento,Porcentaje_Bonificacion_Teorico,Desc_Agrupacion_Rubro_Bain,Desc_Agrup_Reducida_Rubro_Bain,Modelo,Plantilla_Origen,Version_Origen,Medio,Marca,Condicion_Soft,Segmento,Sub_segmento_corporativo_2,Operatoria_no_Newland,No_newland,Adm_Posnet,Adm_Visa_Argentina,Adm_Prisma,tipo_recambio,Prioridad_por_volumen,Prioridad_por_rubro,Tipo_Utilidad_Tecnologia,Q_tx_Aprob_UltM,Q_tx_Rech_UltM,Prop_Q_tx_Rech_UltM,Q_tx_Aprob_Ult15D,Q_tx_Rech_Ult15D,Prop_Q_tx_Rech_Ult15D,Q_tx_Aprob_Ult10D,Q_tx_Rech_Ult10D,Prop_Q_tx_Rech_Ult10D,Q_tx_Aprob_Ult5D,Q_tx_Rech_Ult5D,Prop_Q_tx_Rech_Ult5D,Q_tx_Aprob_Ult4D,Prop_Q_tx_Rech_Ult4D,Q_tx_Aprob_Ult3D,Q_tx_Rech_Ult3D,Prop_Q_tx_Rech_Ult3D,Q_tx_Aprob_Ult2D,Prop_Q_tx_Rech_Ult2D,Q_tx_Aprob_UltD,Q_tx_Rech_UltD,Prop_Q_tx_Rech_UltD,Vol_tx_Aprob_UltM,Vol_tx_Rech_UltM,Prop_Vol_tx_Rech_UltM,Vol_tx_Aprob_Ult15D,Vol_tx_Rech_Ult15D,Prop_Vol_tx_Rech_Ult15D,Vol_tx_Aprob_Ult10D,Vol_tx_Rech_Ult10D,Prop_Vol_tx_Rech_Ult10D,Vol_tx_Aprob_Ult5D,Vol_tx_Rech_Ult5D,Prop_Vol_tx_Rech_Ult5D,Vol_tx_Aprob_Ult4D,Vol_tx_Rech_Ult4D,Prop_Vol_tx_Rech_Ult4D,Vol_tx_Aprob_Ult3D,Vol_tx_Rech_Ult3D,Prop_Vol_tx_Rech_Ult3D,Vol_tx_Aprob_Ult2D,Vol_tx_Rech_Ult2D,Prop_Vol_tx_Rech_Ult2D,Vol_tx_Aprob_UltD,Vol_tx_Rech_UltD,Prop_Vol_tx_Rech_UltD,Clase
0,202102,1174,4,0,5812,6,920,4,7.0,0,1,138,27,880.00,880.00,0.00,7,3,2,61,51,2,1,1,3,1,0,0,0,0,0,1,3,3,1,831,32,0.0371,436,17,0.0375,297,6,0.0198,144,2,0.0137,127,0.0155,111,2,0.0177,81,0.0241,48,0,0.0000,2268673.0000,88370.0000,0.03749189,1178055.0000,49750.0000,0.04051946,818765.0000,17410.0000,0.02082100,411729.0000,7500.0000,0.01788998,353879.0000,7500.0000,0.02075383,320029.0000,7500.0000,0.02289874,251779.0000,7500.0000,0.02892637,150359.0000,0.0000,0E-8,1
2,202103,1235,4,0,5411,21,1989,4,67.0,0,0,52,27,880.00,880.00,0.00,14,8,2,61,54,1,1,1,3,7,0,0,0,0,0,0,4,2,1,68,2,0.0286,68,2,0.0286,68,2,0.0286,13,0,0.0000,0,0.0000,0,0,0.0000,0,0.0000,0,0,0.0000,60498.0000,3215.0000,0.05046066,60498.0000,3215.0000,0.05046066,60498.0000,3215.0000,0.05046066,10931.0000,0.0000,0E-8,0.0000,0.0000,0E-8,0.0000,0.0000,0E-8,0.0000,0.0000,0E-8,0.0000,0.0000,0E-8,1


## One-Hot-Enconder

In [ ]:
#from sklearn.preprocessing import OneHotEncoder

In [ ]:
#df1= pd.get_dummies(df1, columns=["Tipo_inactividad","serv_tecnico","Provincia","tipo_tecnologia","estado_terminal","superestado_terminal","Modelo","Plantilla_Origen","Medio","Condicion_Soft","tipo_recambio","Tipo_Utilidad_Tecnologia"], drop_first=True)

In [ ]:
df1.head(5)

In [ ]:
#ct = ColumnTransformer([('one-hot-encoder', OneHotEncoder(drop='first'), ["Tipo_inactividad","serv_tecnico","Provincia","Localidad","tipo_tecnologia","estado_terminal","superestado_terminal","CUIT_VIP","Modelo","Plantilla_Origen","Version_Origen","Medio","Marca","Condicion_Soft","Segmento","tipo_recambio","Tipo_Utilidad_Tecnologia"])], remainder='passthrough')
#ct.fit_transform(df1) # No funciona muy bien

In [ ]:
# Creo una instancia para One-hot-encoder
# enc = OneHotEncoder(handle_unknown='ignore')
#campos_no_numericos= ["Tipo_inactividad","serv_tecnico","Provincia","Localidad","tipo_tecnologia","estado_terminal","superestado_terminal","CUIT_VIP","Modelo","Plantilla_Origen","Version_Origen","Medio","Marca","Condicion_Soft","Segmento","tipo_recambio","Tipo_Utilidad_Tecnologia"]
#df1[campos_no_numericos] = df1[campos_no_numericos].apply(OneHotEncoder(handle_unknown="ignore").fit_transform)

### Paso todo a numerico y reviso el formato (por las dudas)

In [26]:
df1= df1.astype('float64')
dataTypeSeries = df1.dtypes
print(dataTypeSeries)

Periodo                   float64
Antiguedad                float64
Tipo_inactividad          float64
serv_tecnico              float64
cod_rubro                 float64
                           ...   
Prop_Vol_tx_Rech_Ult2D    float64
Vol_tx_Aprob_UltD         float64
Vol_tx_Rech_UltD          float64
Prop_Vol_tx_Rech_UltD     float64
Clase                     float64
Length: 82, dtype: object


In [ ]:
#df1.to_csv("Chequeo.csv", index=False, header=False)
#df1.to_excel("Chek.xlsx")

### Grupo de Test & Entrenamientos

In [27]:
from sklearn.model_selection import train_test_split
df_train = df1[(df1['Periodo'] >= 202008) & (df1['Periodo'] <= 202102)]
df_test = df1[(df1['Periodo'] >= 202103) & (df1['Periodo'] <= 202104)]

### Variables X e Y: Test y Entrenamientos

In [28]:
X_train=df_train.drop(columns=['Clase']).values
X_test=df_test.drop(columns=['Clase']).values
y_train = df_train["Clase"].values
y_test = df_test["Clase"].values

# Escalado de variables

In [376]:
#from sklearn.preprocessing import StandardScaler
#sc_X = StandardScaler()
#X_train = sc_X.fit_transform(X_train)
#X_test = sc_X.transform(X_test)

# Reducir la dimensión del dataset con PCA

In [ ]:
#from sklearn.decomposition import PCA

In [ ]:
# Me quedo con los 40 principales elementos
#pca = PCA(n_components = 60)
#X_train = pca.fit_transform(X_train)
#X_test = pca.transform(X_test)
#explained_variance = pca.explained_variance_ratio_

## CORRO EL MODELO

In [29]:
#from sklearn.linear_model import LogisticRegression
#classifier = LogisticRegression(random_state = 0)
#classifier.fit(X_train, y_train)
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

C:\Users\eeliano\AnacondaNuevo\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:02:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [30]:
valores=pd.Series(classifier.feature_importances_)
columnas=pd.Series(df_train.drop(columns=["Clase"]).columns)
pd.set_option('display.max_rows', None)
var_imp=pd.concat([columnas,valores],axis=1)
var_imp.columns=['var','val']
var_imp.sort_values(by='val',ascending=False,inplace=True)
print(var_imp)
pd.set_option('display.max_rows', 60)

                                var       val
19                 Plantilla_Origen  0.172715
18                           Modelo  0.043146
26            Operatoria_no_Newland  0.038916
23                   Condicion_Soft  0.036601
7                   tipo_tecnologia  0.026853
21                            Medio  0.026066
0                           Periodo  0.025835
36                   Q_tx_Rech_UltM  0.025796
32            Prioridad_por_volumen  0.023551
35                  Q_tx_Aprob_UltM  0.022219
57                Vol_tx_Aprob_UltM  0.021200
20                   Version_Origen  0.018642
3                      serv_tecnico  0.015459
22                            Marca  0.014408
24                         Segmento  0.013907
15  Porcentaje_Bonificacion_Teorico  0.011180
9                          CUIT_VIP  0.010768
38                Q_tx_Aprob_Ult15D  0.010712
5                         Provincia  0.010689
2                  Tipo_inactividad  0.009870
1                        Antigueda

valores=pd.Series(classifier.feature_importances_)
columnas=pd.Series(df_train.drop(columns=["Clase"]).columns)
pd.set_option('display.max_rows', None)
var_imp=pd.concat([columnas,valores],axis=1)
var_imp.columns=['var','val']
var_imp.sort_values(by='val',ascending=False,inplace=True)
print(var_imp)
pd.set_option('display.max_rows', 58)

In [31]:
y_pred  = classifier.predict(X_test)

C:\Users\eeliano\AnacondaNuevo\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [32]:
y_test.mean()

0.49754983586279083

In [33]:
y_pred.mean()

0.439269232599077

In [34]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [35]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,y_pred)

0.7060206820763107

In [36]:
def balanced_accuracy(cm):
  sensibilidad = cm[1,1]/(cm[1,1]+cm[0,1])
  especificidad = cm[0,0]/(cm[1,0]+cm[0,0])
  return [sensibilidad, especificidad, (sensibilidad + especificidad)/2]

metricas = balanced_accuracy(cm)

print('Matriz de confusion del modelo es:')
print(cm)
print('Sensibilidad del modelo es de: {}'.format(round(metricas[0],4)))  
print('Especificidad del modelo es de: {}'.format(round(metricas[1],4)))  
print('BA del modelo es de: {}'.format(round(metricas[2],4)))

Matriz de confusion del modelo es:
[[8087 2474]
 [3699 6759]]
Sensibilidad del modelo es de: 0.732
Especificidad del modelo es de: 0.6862
BA del modelo es de: 0.7091


In [42]:
scores = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 3, scoring='roc_auc')
scores.std()

C:\Users\eeliano\AnacondaNuevo\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:04:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\eeliano\AnacondaNuevo\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:05:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\eeliano\AnacondaNuevo\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:05:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.0022918077475354262

In [43]:
scores.mean()

0.7402856731426063